In [274]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import division
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
%matplotlib inline

In [301]:
pd.set_option('display.max_columns', 100)

In [302]:
df = pd.read_json('data.json', convert_dates=['approx_payout_date','event_created',\
                                               'event_end','event_published',\
                                               'event_start','sale_duration',\
                                               'sale_duration2','user_created',\
                                               'event_published'])

In [303]:
df.head() 

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,2010-02-13 12:00:00,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,2010-01-06 01:01:46,2010-02-08 12:00:00,1.263110e+09,2010-02-08 02:00:00,0,0.00,0,1.0,0,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBRITY WEEKEN...",60,0,0,527017,,0.0,Party Starz Ent & Diverse Int'l Group,0.0,,,[],29.0,33,1,"[{u'event_id': 527017, u'cost': 25.0, u'availa...",36,2009-11-30 20:45:50,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,premium,2011-02-03 08:00:00,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,2010-12-31 21:57:50,2011-01-29 08:00:00,1.293833e+09,2011-01-28 23:00:00,0,868.02,0,0.0,1,n,Winthrop RUF Winter Getaway,27,23,1,786878,"<p>Since 1987, RUF has ministered to students ...",0.0,RUF at Winthrop University,12.0,RUF,CHECK,"[{u'name': u'RUF', u'created': u'2010-10-01 01...",28.0,28,0,"[{u'event_id': 786878, u'cost': 35.0, u'availa...",149,2010-08-04 17:26:16,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC
2,premium,2011-01-28 00:00:00,2601,8,US,USD,1.0,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,2010-11-30 04:22:36,2011-01-23 00:00:00,1.291092e+09,2011-01-22 16:30:00,0,3500.00,0,NaN,0,y,DRDP (2010) Teacher Training,28,19,4,787337,"<p><a href=""http://www.desiredresults.com"">www...",0.0,University Preparation School in collaboration...,0.0,University Preparation School,CHECK,"[{u'name': u'Danielle Severn', u'created': u'2...",54.0,54,0,"[{u'event_id': 787337, u'cost': 93.51, u'avail...",214,2010-04-29 16:43:08,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA
3,premium,2014-01-06 00:00:00,12347,6,IE,EUR,1.0,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,2013-02-12 15:06:10,2014-01-01 00:00:00,1.360683e+09,2013-02-12 21:00:00,0,1167.35,0,0.0,1,y,ITTA Affiliation 2013,21,39,16,885645,,0.0,,0.0,,ACH,"[{u'name': u'', u'created': u'2010-11-09 01:10...",0.0,0,0,"[{u'event_id': 885645, u'cost': 25.0, u'availa...",889,2010-09-07 14:35:02,3,,None,NaN,NaN,None,None
4,premium,2011-02-17 00:00:00,2417,11,US,USD,0.0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,2010-12-10 15:24:26,2011-02-12 00:00:00,1.291995e+09,2011-02-11 16:00:00,1,2313.15,0,0.0,0,y,Self Defense for Writers and Filmmakers: For A...,66,30,0,1114349,"<p><font face=""Arial""><font size=""2""><strong>T...",0.0,Arts & Business Council of Greater Boston,0.0,Arts and Business Council or Greater Boston,CHECK,[{u'name': u'Arts and Business Council or Grea...,63.0,63,0,"[{u'event_id': 1114349, u'cost': 150.0, u'avai...",35,2010-11-05 19:07:45,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA


In [304]:
df_new = df.loc[df.acct_type.isin(['premium','fraudster_event','fraudster','fraudster_att']), :]

In [305]:
df_new['fraud'] = 0
df_new.loc[df.acct_type.isin(['fraudster_event', 'fraudster', 'fraudster_att']), 'fraud'] = 1

/Users/lihuama/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [306]:
df_new.head()

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,fraud
0,fraudster_event,2010-02-13 12:00:00,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,2010-01-06 01:01:46,2010-02-08 12:00:00,1.263110e+09,2010-02-08 02:00:00,0,0.00,0,1.0,0,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBRITY WEEKEN...",60,0,0,527017,,0.0,Party Starz Ent & Diverse Int'l Group,0.0,,,[],29.0,33,1,"[{u'event_id': 527017, u'cost': 25.0, u'availa...",36,2009-11-30 20:45:50,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL,1
1,premium,2011-02-03 08:00:00,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,2010-12-31 21:57:50,2011-01-29 08:00:00,1.293833e+09,2011-01-28 23:00:00,0,868.02,0,0.0,1,n,Winthrop RUF Winter Getaway,27,23,1,786878,"<p>Since 1987, RUF has ministered to students ...",0.0,RUF at Winthrop University,12.0,RUF,CHECK,"[{u'name': u'RUF', u'created': u'2010-10-01 01...",28.0,28,0,"[{u'event_id': 786878, u'cost': 35.0, u'availa...",149,2010-08-04 17:26:16,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC,0
2,premium,2011-01-28 00:00:00,2601,8,US,USD,1.0,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,2010-11-30 04:22:36,2011-01-23 00:00:00,1.291092e+09,2011-01-22 16:30:00,0,3500.00,0,NaN,0,y,DRDP (2010) Teacher Training,28,19,4,787337,"<p><a href=""http://www.desiredresults.com"">www...",0.0,University Preparation School in collaboration...,0.0,University Preparation School,CHECK,"[{u'name': u'Danielle Severn', u'created': u'2...",54.0,54,0,"[{u'event_id': 787337, u'cost': 93.51, u'avail...",214,2010-04-29 16:43:08,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA,0
3,premium,2014-01-06 00:00:00,12347,6,IE,EUR,1.0,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,2013-02-12 15:06:10,2014-01-01 00:00:00,1.360683e+09,2013-02-12 21:00:00,0,1167.35,0,0.0,1,y,ITTA Affiliation 2013,21,39,16,885645,,0.0,,0.0,,ACH,"[{u'name': u'', u'created': u'2010-11-09 01:10...",0.0,0,0,"[{u'event_id': 885645, u'cost': 25.0, u'availa...",889,2010-09-07 14:35:02,3,,None,NaN,NaN,None,None,0
4,premium,2011-02-17 00:00:00,2417,11,US,USD,0.0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,2010-12-10 15:24:26,2011-02-12 00:00:00,1.291995e+09,2011-02-11 16:00:00,1,2313.15,0,0.0,0,y,Self Defense for Writers and Filmmakers: For A...,66,30,0,1114349,"<p><font face=""Arial""><font size=""2""><strong>T...",0.0,Arts & Business Council of Greater Boston,0.0,Arts and Business Council or Greater Boston,CHECK,[{u'name': u'Arts and Business Council or Grea...,63.0,63,0,"[{u'event_id': 1114349, u'cost': 150.0, u'avai...",35,2010-11-05 19:07:45,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA,0


In [307]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13666 entries, 0 to 14336
Data columns (total 45 columns):
acct_type             13666 non-null object
approx_payout_date    13666 non-null datetime64[ns]
body_length           13666 non-null int64
channels              13666 non-null int64
country               13590 non-null object
currency              13666 non-null object
delivery_method       13650 non-null float64
description           13666 non-null object
email_domain          13666 non-null object
event_created         13666 non-null datetime64[ns]
event_end             13666 non-null datetime64[ns]
event_published       13567 non-null float64
event_start           13666 non-null datetime64[ns]
fb_published          13666 non-null int64
gts                   13666 non-null float64
has_analytics         13666 non-null int64
has_header            8469 non-null float64
has_logo              13666 non-null int64
listed                13666 non-null object
name                  136

In [308]:
df_new.fraud.value_counts()

0    12373
1     1293
Name: fraud, dtype: int64

In [309]:
df_new.fraud.value_counts()[1]/len(df_new.fraud)

0.094614371432752822

In [310]:
df_new['previous_payouts_total'] = df_new.previous_payouts.apply(len)

/Users/lihuama/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [312]:
feature_cols = ['delivery_method', 'num_payouts', 'org_facebook', 'org_twitter', 'sale_duration', \
                'previous_payouts_total']

In [313]:
X = df_new[feature_cols]
y = df_new.fraud

In [314]:
df_new = pd.concat([X,y], axis = 1)

In [315]:
df_new = df_new.dropna().copy()

In [316]:
X_new = df_new[feature_cols]
y_new = df_new.fraud

In [317]:
X_train, X_test, y_train, y_test = train_test_split(X_new,y_new)

In [318]:
rm = RandomForestClassifier(n_estimators=100)

In [319]:
rm.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [320]:
rm.score(X_test, y_test)

0.97950089126559714

In [321]:
roc_auc_score(y_test, rm.predict_proba(X_test)[:, 1])

0.96601857641530486

In [322]:
confusion_matrix(y_test, rm.predict(X_test))

array([[3050,   23],
       [  46,  247]])

In [323]:
c = confusion_matrix(y_test, rm.predict(X_test))

In [324]:
precision_score(y_test, rm.predict(X_test))

0.91481481481481486

In [325]:
recall_score = c[1,1]/(c[1,1] + c[1,0])

In [326]:
recall_score

0.84300341296928327

In [327]:
def total_revenue(dicts):
    revenues = []
    for d in dicts:
        c = d['cost']
        sold = d['quantity_sold']
        revenue = c * float(sold)
        revenues.append(revenue)
    return sum(revenues)

def lifetime_value(revenue, num_days):
    if num_days != 0:
        periods = float(num_days)/365
        GC = revenue/periods
        CLV = GC * (1/1+0.04)
        return CLV
    
df['Revenue'] = df['ticket_types'].apply(lambda x: total_revenue(x))
df['Lifetime Value'] = df.apply(lambda x: lifetime_value(x['Revenue'], x['user_age']), axis = 1)

In [328]:
df_new_value = df_new.copy()
df_new_value['ticket_types'] = df.ticket_types
df_new_value['user_age'] = df.user_age

In [329]:
df_new_value.head()

,delivery_method,num_payouts,org_facebook,org_twitter,sale_duration,previous_payouts_total,fraud,ticket_types,user_age
0,0.0,0,0.0,0.0,29.0,0,1,"[{u'event_id': 527017, u'cost': 25.0, u'availa...",36
1,1.0,1,0.0,12.0,28.0,49,0,"[{u'event_id': 786878, u'cost': 35.0, u'availa...",149
2,1.0,4,0.0,0.0,54.0,36,0,"[{u'event_id': 787337, u'cost': 93.51, u'avail...",214
3,1.0,16,0.0,0.0,0.0,21,0,"[{u'event_id': 885645, u'cost': 25.0, u'availa...",889
4,0.0,0,0.0,0.0,63.0,49,0,"[{u'event_id': 1114349, u'cost': 150.0, u'avai...",35


In [330]:
df_new_value['Revenue'] = df_new_value['ticket_types'].apply(lambda x: total_revenue(x))
df_new_value['Lifetime Value'] = df_new_value.apply(lambda x: lifetime_value(x['Revenue'], x['user_age']), axis = 1)

In [331]:
df_new_value.head(3)

,delivery_method,num_payouts,org_facebook,org_twitter,sale_duration,previous_payouts_total,fraud,ticket_types,user_age,Revenue,Lifetime Value
0,0.0,0,0.0,0.0,29.0,0,1,"[{u'event_id': 527017, u'cost': 25.0, u'availa...",36,0.00,0.000000
1,1.0,1,0.0,12.0,28.0,49,0,"[{u'event_id': 786878, u'cost': 35.0, u'availa...",149,875.00,2229.194631
2,1.0,4,0.0,0.0,54.0,36,0,"[{u'event_id': 787337, u'cost': 93.51, u'avail...",214,4488.48,7961.808449


In [332]:
df_new_value['predict'] = rm.predict(X_new)

In [333]:
(df_new_value.fraud == df_new_value.predict).sum()

13308